Retrieve the token from the environment and request information about it.
This calls two token APIs, one to retrieve information about the user identified by the token, and one to retrieve information about the token itself.

In [ ]:
import logging
import os
import time
from urllib.parse import urljoin

import requests

token = os.getenv('ACCESS_TOKEN', None)

if token is None:
    logging.fatal('You have no token.')
    
base_url = os.getenv('EXTERNAL_URL')
user_info_url = urljoin(base_url, '/auth/api/v1/user-info')
r = requests.get(user_info_url, headers={'Authorization': f'bearer {token}'})
data = r.json()

print("Your username is", data['username'])
print("Your name is", (data.get('name', data['username'])))
print("Your numeric UID is", data['uid'])

token_info_url = urljoin(base_url, '/auth/api/v1/token-info')
r = requests.get(token_info_url, headers={'Authorization': f'bearer {token}'})
data = r.json()

iat = time.gmtime(int(data['created']))
print("Your token was issued at UTC:", time.asctime(iat))

exp = time.gmtime(int(data['expires']))
print("Your token expires at UTC:   ", time.asctime(exp))

current_time = time.gmtime()
print("The time is currently UTC:   ", time.asctime(current_time))

if current_time < iat:
    logging.error("Your token isn't valid yet.")
    
if iat < current_time and current_time < exp:
    print("Your token is VALID.")

if current_time > exp:
    logging.error("Your token is expired!")
    raise RuntimeError(f"Your token expired at UTC: {time.asctime(exp)}")